# Just to test a manual implementation of BM25 retriever

In [1]:
import numpy as np
from typing import List, Tuple
from collections import Counter
import math

In [2]:
class SimpleBM25:
    """
    Simple BM25 implementation from scratch.

    BM25 Formula:
    score(D,Q) = Σ IDF(qi) * (f(qi,D) * (k1+1)) / (f(qi,D) + k1 * (1 - b + b * |D| / avgdl))

    Where:
    - D: document
    - Q: query
    - qi: query term i
    - f(qi,D): frequency of qi in D
    - |D|: length of document D
    - avgdl: average document length
    - k1, b: tuning parameters
    - IDF(qi): inverse document frequency of qi
    """

    def __init__(self, k1: float = 1.5, b: float = 0.75):
        """
        Initialize BM25.

        Args:
            k1: Term frequency saturation parameter (typically 1.2-2.0)
            b: Length normalization parameter (typically 0.75)
        """
        self.k1 = k1
        self.b = b
        self.documents = []
        self.doc_lengths = []
        self.avgdl = 0
        self.doc_freqs = []  # Term frequencies per document
        self.idf_scores = {}  # IDF scores for each term
        self.vocab = set()

        print(f"SimpleBM25 initialized (k1={k1}, b={b})")

    def _tokenize(self, text: str) -> List[str]:
        """Simple tokenization (lowercase + split)."""
        return text.lower().split()

    def _compute_idf(self):
        """
        Compute IDF scores for all terms.

        IDF(qi) = log((N - df(qi) + 0.5) / (df(qi) + 0.5))
        Where N = total documents, df(qi) = documents containing qi
        """
        N = len(self.documents)

        # Count document frequency for each term
        df = Counter()
        for doc_terms in self.doc_freqs:
            df.update(doc_terms.keys())

        # Compute IDF
        for term in self.vocab:
            df_term = df[term]
            idf = math.log((N - df_term + 0.5) / (df_term + 0.5) + 1.0)
            self.idf_scores[term] = idf

    def index(self, documents: List[str]):
        """
        Index documents for BM25 scoring.

        Args:
            documents: List of document strings
        """
        self.documents = documents
        self.doc_lengths = []
        self.doc_freqs = []

        # Process each document
        for doc in documents:
            tokens = self._tokenize(doc)
            self.doc_lengths.append(len(tokens))

            # Count term frequencies
            term_freqs = Counter(tokens)
            self.doc_freqs.append(term_freqs)
            self.vocab.update(tokens)

        # Compute average document length
        self.avgdl = sum(self.doc_lengths) / len(self.doc_lengths)

        # Compute IDF scores
        self._compute_idf()

        print(f"✓ Indexed {len(documents)} documents")
        print(f"  Vocabulary size: {len(self.vocab)}")
        print(f"  Avg doc length: {self.avgdl:.1f} tokens")

    def score_document(self, query_terms: List[str], doc_idx: int) -> float:
        """
        Compute BM25 score for a single document.

        Args:
            query_terms: List of query terms
            doc_idx: Document index

        Returns:
            BM25 score
        """
        score = 0.0
        doc_len = self.doc_lengths[doc_idx]
        doc_freqs = self.doc_freqs[doc_idx]

        for term in query_terms:
            if term not in self.vocab:
                continue

            # Get term frequency in document
            f = doc_freqs.get(term, 0)

            # Get IDF
            idf = self.idf_scores[term]

            # Compute BM25 component for this term
            numerator = f * (self.k1 + 1)
            denominator = f + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)

            score += idf * (numerator / denominator)

        return score

    def search(self, query: str, k: int = 5) -> List[Tuple[int, float]]:
        """
        Search for top-k documents.

        Args:
            query: Query string
            k: Number of results to return

        Returns:
            List of (doc_index, score) tuples
        """
        query_terms = self._tokenize(query)

        # Score all documents
        scores = []
        for idx in range(len(self.documents)):
            score = self.score_document(query_terms, idx)
            scores.append((idx, score))

        # Sort by score (descending) and return top-k
        scores.sort(key=lambda x: x[1], reverse=True)
        return scores[:k]

In [3]:
# Test documents
docs = [
    "The movie is about dreams and reality",
    "A science fiction film with time travel",
    "Romantic comedy set in New York",
    "Inception is a movie about dream heist",
    "Christopher Nolan directed Inception"
]

# Create and index
bm25 = SimpleBM25()
bm25.index(docs)

# Test searches
test_queries = [
    "inception dream",
    "time travel science fiction",
    "new york comedy"
]

for query in test_queries:
    print(f"\nQuery: '{query}'")
    results = bm25.search(query, k=3)
    for idx, score in results:
        print(f"  Score {score:.4f}: {docs[idx]}")

SimpleBM25 initialized (k1=1.5, b=0.75)
✓ Indexed 5 documents
  Vocabulary size: 26
  Avg doc length: 6.2 tokens

Query: 'inception dream'
  Score 2.1376: Inception is a movie about dream heist
  Score 1.0418: Christopher Nolan directed Inception
  Score 0.0000: The movie is about dreams and reality

Query: 'time travel science fiction'
  Score 5.2409: A science fiction film with time travel
  Score 0.0000: The movie is about dreams and reality
  Score 0.0000: Romantic comedy set in New York

Query: 'new york comedy'
  Score 4.2201: Romantic comedy set in New York
  Score 0.0000: The movie is about dreams and reality
  Score 0.0000: A science fiction film with time travel
